In [5]:
from utils import *
import time
from sklearn.svm import LinearSVC, SVC
from copy import deepcopy
import numpy as np
import scipy.io as sio

import math
import tensorflow as tf
from scipy import signal
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import mutual_info_classif

'''
from mutual_information import mutual_information 
        https://gist.github.com/elsonidoq/4230222
        doesn't make any deffrence in output    
'''
import math
import time
import pyriemann.utils.mean as rie_mean
from scipy.signal import hilbert
from scipy import linalg

import pandas as pd

#get_data.py loads the dataset for a spceific subject, and the option of whether the training data or testing data
#utils.py have the underlying functions such as extract features function and the filter bank that allow for the FBCSP class to work
#the FBCSP class showcases the different function that work hand in hand to provied the FBCSP algorithm

#Figure out how to utilize the FBCSP Class
#Include code from Utils.py and COMMENTS
#Test classifier
#Retrieve Random Data Pointt

In [26]:
#GETDATA FUNCTION

def get_data(subject,training,PATH):
    '''
    Loads the dataset 2a of the BCI Competition IV
    available on http://bnci-horizon-2020.eu/database/data-sets

    Keyword arguments:
    subject -- number of subject in [1, .. ,9]
    training -- if True, load training data
                if False, load testing data

    Return: data_return 	numpy matrix 	size = NO_valid_trial x 22 x 1750
    class_return 	numpy matrix 	size = NO_valid_trial
    '''
    NO_channels = 22
    NO_tests = 6*48
    Window_Length = 7*250

    class_return = np.zeros(NO_tests)
    data_return = np.zeros((NO_tests,NO_channels,Window_Length))

    NO_valid_trial = 0
    print("PATH : ",PATH,"  subject : ",subject)
    if training:
        a = sio.loadmat(PATH+'A0'+str(subject)+'T.mat')
    else:
        a = sio.loadmat(PATH+'A0'+str(subject)+'E.mat')
    a_data = a['data']
    
    print(a)
    
    for ii in range(0,a_data.size):
        a_data1 = a_data[0,ii]
        a_data2=[a_data1[0,0]]
        a_data3=a_data2[0]
        a_X= a_data3[0]
        a_trial= a_data3[1]
        a_y = a_data3[2]
        a_fs= a_data3[3]
        a_classes= a_data3[4]
        a_artifacts= a_data3[5]
        a_gender= a_data3[6]
        a_age= a_data3[7]
        for trial in range(0,a_trial.size):
            if(a_artifacts[trial]==0):
                data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+Window_Length),:22])
                class_return[NO_valid_trial] = int(a_y[trial])
                NO_valid_trial +=1
                
    sorted_indices = np.argsort(class_return[0:NO_valid_trial])
    sorted_data_return = data_return[sorted_indices]
    sorted_class_return = class_return[sorted_indices]


    return sorted_data_return, sorted_class_return

In [27]:
[sorted_data,sorted_class] = get_data(3,True,'C:/Users/DLSU/Desktop/Multiple Classes FBCSP/datasets/')

PATH :  C:/Users/DLSU/Desktop/Multiple Classes FBCSP/datasets/   subject :  3
{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Thu Aug  7 13:07:29 2014', '__version__': '1.0', '__globals__': [], 'data': array([[array([[(array([[  6.49414062,  14.40429688,  12.40234375, ...,   4.39453125,
                          4.39453125,   0.9765625 ],
                       [  8.05664062,  15.08789062,  13.81835938, ...,   7.32421875,
                          1.953125  ,   2.9296875 ],
                       [ 14.84375   ,  20.55664062,  20.01953125, ...,   8.30078125,
                          9.27734375,   4.39453125],
                       ...,
                       [-12.54882812,  -7.2265625 ,  -9.08203125, ...,  -1.953125  ,
                         -6.8359375 , -11.23046875],
                       [-16.55273438, -12.93945312, -13.23242188, ...,  -5.37109375,
                         -7.8125    ,  -6.8359375 ],
                       [ -7.56835938,  -4.8828125 ,  -6.54

In [20]:
sorted_data

array([[[ 7.27539062e+00, -2.92968750e-01, -6.78710938e+00, ...,
          4.98046875e+00,  6.39648438e+00,  5.56640625e+00],
        [ 4.73632812e+00, -7.42187500e+00, -9.47265625e+00, ...,
          3.95507812e+00,  5.07812500e+00, -5.85937500e-01],
        [ 2.44140625e-01, -7.51953125e+00, -1.39160156e+01, ...,
          5.37109375e+00,  7.71484375e+00,  5.46875000e+00],
        ...,
        [ 1.01074219e+01, -1.46484375e-01, -2.00195312e+00, ...,
         -2.68554688e+00,  7.81250000e-01, -1.95312500e-01],
        [ 1.08886719e+01, -2.29492188e+00, -3.27148438e+00, ...,
         -2.19726562e+00,  1.85546875e+00,  1.80664062e+00],
        [ 9.22851562e+00,  2.44140625e-01,  1.12304688e+00, ...,
         -2.83203125e+00,  2.44140625e-01, -2.49023438e+00]],

       [[-1.25000000e+01,  4.88281250e-02, -1.84570312e+01, ...,
          7.76367188e+00,  1.36718750e+00,  1.25976562e+01],
        [-6.98242188e+00,  5.56640625e+00, -1.25000000e+01, ...,
          1.29882812e+01, -5.22460938e

In [21]:
sorted_class

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4.

In [24]:
savedict = {
    'TrData' : sorted_data,
    'TrLabel' : sorted_class,
}

In [25]:
sio.savemat('test.mat', savedict)